In [159]:
## Dependencies
import spotipy
import requests
import pandas as pd
import tensorflow as tf
from spotipy.oauth2 import SpotifyOAuth
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [160]:
url  = "https://accounts.spotify.com/api/token"

headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

data = {
    "grant_type":"client_credentials",
    "client_id":"ca7701006fdc4cd7bc7291766b4f96d9",
    "client_secret":"b976e76921704f6081f8a871b9bb81b4"
}

response = requests.post(url=url, headers=headers, data=data)

if response.status_code == 200:
    # Request was successful, parse the JSON response
    token_data = response.json()
    access_token = token_data.get("access_token")
    print("Access Token:", access_token)
else:
    # Request failed, print error message and status code
    print("Error:", response.status_code, response.text)

Access Token: BQBwYD5cUCmofHXuPY9iFHQxBJSAwG6bUvUHejKi1CXxACL55j1Ljf-Sh-hsPC0E1bWHID8TwSAhY3pVphQx0GETyHg4-gI85m3WGTp1Q7bwVsyqfkQ


In [161]:
# CHANGE THIS URL TO CHANGE THE PLAYLIST IT QUERIES
url = "https://api.spotify.com/v1/playlists/2fmTTbBkXi8pewbUvG3CeZ"

headers = {
    'Authorization': 'Bearer %s' %(access_token),
    'Content-Type': 'application/json'
}

artist_reponse = requests.get(url=url, headers=headers).json()

artist_reponse

{'collaborative': False,
 'description': 'Rewind and rediscover big songs of 2020. Cover: The Weeknd',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/2fmTTbBkXi8pewbUvG3CeZ'},
 'followers': {'href': None, 'total': 68791},
 'href': 'https://api.spotify.com/v1/playlists/2fmTTbBkXi8pewbUvG3CeZ',
 'id': '2fmTTbBkXi8pewbUvG3CeZ',
 'images': [{'height': None,
   'url': 'https://i.scdn.co/image/ab67706c0000bebb7eb0db8793e20fa43df1d2af',
   'width': None}],
 'name': 'Top Hits of 2020',
 'owner': {'display_name': 'Spotifyn',
  'external_urls': {'spotify': 'https://open.spotify.com/user/nick170708%21'},
  'href': 'https://api.spotify.com/v1/users/nick170708%21',
  'id': 'nick170708!',
  'type': 'user',
  'uri': 'spotify:user:nick170708%21'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MjEyNyw0ZjEyMWQzOWVmNzRkYWUwNWMwYmY5ODM0ZjlmODM2MzcxN2NiODU1',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/2fmTTbBkXi8pewbUvG3CeZ/tracks?offset=0&limit=100',
  'items':

In [162]:
playlist_track_ids = []

tracks = artist_reponse['tracks']['items']

track_ids = [track['track']['id'] for track in tracks]

for track_id in track_ids:
    playlist_track_ids.append(track_id)

print(len(playlist_track_ids))

100


In [163]:
track_info = []

query_url = "https://api.spotify.com/v1/tracks/"

for track in playlist_track_ids:
    
    headers = {
    'Authorization': 'Bearer %s' %(access_token),
    'Content-Type': 'application/json'
    }

    track_reponse = requests.get(url=query_url + track, headers=headers).json()

    track_info.append(track_reponse)

track_info

[{'album': {'album_type': 'single',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ'},
     'href': 'https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ',
     'id': '1Xyo4u8uXC1ZmMpatF05PJ',
     'name': 'The Weeknd',
     'type': 'artist',
     'uri': 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ'}],
   'available_markets': ['KR'],
   'external_urls': {'spotify': 'https://open.spotify.com/album/2ZfHkwHuoAZrlz7RMj0PDz'},
   'href': 'https://api.spotify.com/v1/albums/2ZfHkwHuoAZrlz7RMj0PDz',
   'id': '2ZfHkwHuoAZrlz7RMj0PDz',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab67616d0000b273c464fabb4e51b72d657f779a',
     'width': 640},
    {'height': 300,
     'url': 'https://i.scdn.co/image/ab67616d00001e02c464fabb4e51b72d657f779a',
     'width': 300},
    {'height': 64,
     'url': 'https://i.scdn.co/image/ab67616d00004851c464fabb4e51b72d657f779a',
     'width': 64}],
   'name': 'Blinding Lights',
   'release_date'

In [164]:
playlist_track_ids = []
song_names = []
artists = []
durations = []
release_dates = []
albums = []
popularity = []

for entry in track_info:
    if 'id' in entry and 'name' in entry and 'album' in entry:
        playlist_track_ids.append(entry['id'])
        song_names.append(entry['name'])
        if 'album' in entry and 'artists' in entry['album'] and entry['album']['artists']:
            artists.append(entry['album']['artists'][0]['name'])
        else:
            artists.append('Unknown Artist')
        durations.append(entry.get('duration_ms', 0))
        release_dates.append(entry['album'].get('release_date', 'Unknown'))
        albums.append(entry['album'].get('name', 'Unknown Album'))
        popularity.append(entry.get('popularity', 0))

In [165]:
track_audio_analysis = []

query_url = "https://api.spotify.com/v1/audio-features/"

for track in playlist_track_ids:
    
    headers = {
    'Authorization': 'Bearer %s' %(access_token),
    'Content-Type': 'application/json'
    }

    track_audio_reponse = requests.get(url=query_url + track, headers=headers).json()

    track_audio_analysis.append(track_audio_reponse)

track_audio_analysis

[{'danceability': 0.513,
  'energy': 0.796,
  'key': 1,
  'loudness': -4.075,
  'mode': 1,
  'speechiness': 0.0629,
  'acousticness': 0.00147,
  'instrumentalness': 0.000209,
  'liveness': 0.0938,
  'valence': 0.345,
  'tempo': 171.017,
  'type': 'audio_features',
  'id': '0sf12qNH5qcw8qpgymFOqD',
  'uri': 'spotify:track:0sf12qNH5qcw8qpgymFOqD',
  'track_href': 'https://api.spotify.com/v1/tracks/0sf12qNH5qcw8qpgymFOqD',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0sf12qNH5qcw8qpgymFOqD',
  'duration_ms': 201573,
  'time_signature': 4},
 {'danceability': 0.613,
  'energy': 0.581,
  'key': 0,
  'loudness': -8.588,
  'mode': 0,
  'speechiness': 0.0424,
  'acousticness': 0.537,
  'instrumentalness': 0.000345,
  'liveness': 0.25,
  'valence': 0.551,
  'tempo': 130.033,
  'type': 'audio_features',
  'id': '4LEK9rD7TWIG4FCL1s27XC',
  'uri': 'spotify:track:4LEK9rD7TWIG4FCL1s27XC',
  'track_href': 'https://api.spotify.com/v1/tracks/4LEK9rD7TWIG4FCL1s27XC',
  'analysis_url': 'ht

In [166]:
track_acousticness = []
track_danceability = []
track_energy = []
track_insturmentalness = []
track_key = []
track_liveness = []
track_loudness = []
track_mode = []
track_spechiness = []
track_tempo = []
track_time_signature = []
track_valence = []

for audio_entry in track_audio_analysis:
    for entry in track_audio_analysis:
        if audio_entry['id'] == entry['id']:
            # Append audio feature information here
            track_acousticness.append(audio_entry['acousticness'])
            track_danceability.append(audio_entry['danceability'])
            track_energy.append(audio_entry['energy'])
            track_insturmentalness.append(audio_entry['instrumentalness'])
            track_key.append(audio_entry['key'])
            track_liveness.append(audio_entry['liveness'])
            track_loudness.append(audio_entry['loudness'])
            track_mode.append(audio_entry['mode'])
            track_spechiness.append(audio_entry['speechiness'])
            track_tempo.append(audio_entry['tempo'])
            track_time_signature.append(audio_entry['time_signature'])
            track_valence.append(audio_entry['valence'])

In [167]:
song_dataframe = pd.DataFrame({
    "song_id":playlist_track_ids,
    "song_name":song_names,
    "album":albums,
    "artist":artists,
    "duration(ms)":durations,
    "release_date":release_dates,
    "acousticness":track_acousticness,
    "danceability":track_danceability,
    "energy":track_energy,
    "insturmentalness":track_insturmentalness,
    "key":track_key,
    "liveness":track_liveness,
    "loudness":track_loudness,
    "mode":track_mode,
    "spechiness":track_spechiness,
    "tempo":track_tempo,
    "time_signature":track_time_signature,
    "valence":track_valence,
    "popularity":popularity
})

song_dataframe

,song_id,song_name,album,artist,duration(ms),release_date,acousticness,danceability,energy,insturmentalness,key,liveness,loudness,mode,spechiness,tempo,time_signature,valence,popularity
0,0sf12qNH5qcw8qpgymFOqD,Blinding Lights,Blinding Lights,The Weeknd,201573,2019-11-29,0.00147,0.513,0.796,0.000209,1,0.0938,-4.075,1,0.0629,171.017,4,0.3450,18
1,4LEK9rD7TWIG4FCL1s27XC,cardigan,folklore,Taylor Swift,239560,2020-07-24,0.53700,0.613,0.581,0.000345,0,0.2500,-8.588,0,0.0424,130.033,4,0.5510,52
2,7igeByaBM0MgGsgXtNxDJ7,positions,positions,Ariana Grande,172324,2020-10-23,0.46800,0.736,0.802,0.000000,0,0.0940,-4.759,1,0.0864,144.005,4,0.6750,11
3,696DnlkuDOXcMAnKlTgXXK,ROXANNE,ROXANNE,Arizona Zervas,163636,2019-10-10,0.05220,0.621,0.601,0.000000,6,0.4600,-5.616,0,0.1480,116.735,5,0.4570,77
4,0nbXyq5TXYPCO7pr3N8S4I,The Box,Please Excuse Me for Being Antisocial,Roddy Ricch,196652,2019-12-06,0.10400,0.896,0.586,0.000000,10,0.7900,-6.687,0,0.0559,116.971,4,0.6420,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2kJu14V7hbZw3I4K8L8SXb,Stupid Love,Stupid Love,Lady Gaga,193530,2020-02-28,0.00146,0.676,0.815,0.000410,8,0.2750,-4.294,1,0.0625,117.973,4,0.7960,31
96,2DEZmgHKAvm41k4J3R2E9Y,Safaera,YHLQMDLG,Bad Bunny,295176,2020-02-29,0.01030,0.607,0.829,0.000000,2,0.1070,-4.074,1,0.3790,96.172,4,0.6850,77
97,59qrUpoplZxbIZxk6X0Bm3,Take You Dancing,Take You Dancing,Jason Derulo,190306,2020-07-22,0.03320,0.789,0.711,0.000000,2,0.0876,-4.248,1,0.0410,112.985,4,0.7530,75
98,1DWZUa5Mzf2BwzpHtgbHPY,Good News,Circles,Mac Miller,342040,2020-01-17,0.85300,0.794,0.320,0.134000,1,0.1120,-12.920,0,0.1730,174.088,4,0.2410,73


In [168]:
song_dataframe.to_csv('Resources/Top100-2020.csv')